<a href="https://colab.research.google.com/github/hedgehogdot/2025-2-Machin-Learning-2/blob/main/ML2_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **(a)** Implement a k-NN algorithm (k = 5) using an iterative method (i.e., using for loop) to classify a single new example. Write down your observations.

In [1]:
import torch
from torchvision import datasets, transforms

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transforms)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transforms)

100%|██████████| 9.91M/9.91M [00:00<00:00, 57.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.68MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.1MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.47MB/s]


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data = train_dataset.data.view(-1, 28*28).float().to(device)
train_labels = train_dataset.targets.to(device)

index = 7843
test_sample = test_dataset.data[index].view(-1, 28*28).float().to(device)  #1차원 벡터로 변환
test_label = test_dataset.targets[index].item()

k = 5
distances = []

for i in range(len(train_data)):
    dist = torch.dist(train_data[i], test_sample, p=2)
    distances.append((dist.item(), train_labels[i].item()))

distances.sort(key=lambda x: x[0])
neighbors = distances[:k]

label_count = {}
for d, lbl in neighbors:
    label_count[lbl] = label_count.get(lbl, 0) + 1

predicted_label = max(label_count, key=label_count.get)

print("Neighbors' labels:", [lbl for _, lbl in neighbors])
print(f"Predicted label: {predicted_label}")
print(f"Actual label: {test_label}")

Neighbors' labels: [6, 6, 6, 6, 6]
Predicted label: 6
Actual label: 6


## **(b)** Implement a k-NN algorithm (k = 5) using the broadcasting concept you learned in the laboratory session to classify a single new example.

In [3]:
# Adjust the dimensions of the data
train_data_gpu = train_data.view(len(train_data), -1).to(device)
train_labels_gpu = train_labels.to(device)
example_gpu = test_sample.view(1, -1).to(device)

# Calculate the Euclidean distance using broadcasting
dist_broadcast = torch.sqrt(torch.sum((train_data_gpu - example_gpu) ** 2, dim=1))

nearest = []

# Loop to select k nearest neighbors
for i in range(len(dist_broadcast)):
    dist = dist_broadcast[i]

    # Store the distance and label based on distance
    if len(nearest) < k:
        nearest.append((dist, train_labels_gpu[i]))
        nearest.sort(key=lambda x: x[0])
    else:
        # If current distance is closer than the farthest neighbor, replace
        if dist < nearest[-1][0]:
            nearest[-1] = (dist, train_labels_gpu[i])
            nearest.sort(key=lambda x: x[0])

# Extract labels of the neighbors
neighbor_labels = [label.item() for _, label in nearest]

# Count the frequency of each label
label_vote = {}
for label in neighbor_labels:
    if label in label_vote:
        label_vote[label] += 1
    else:
        label_vote[label] = 1

# Select the label with the highest frequency
result = max(label_vote, key=label_vote.get)

# Print results
print(f'Neighbor : {neighbor_labels}')
print(f'Classifying result: {result}')

# Retrieve the true label
true_result = test_label
print(f'Actual result : {true_result}')

Neighbor : [6, 6, 6, 6, 6]
Classifying result: 6
Actual result : 6


## **(c)** Now, extend a k-NN algorithm from (b) to perform classification over all digits for all new images at once, using broadcasting (not just a single image). Write down yourobservations. Can you find any issue?



### broadcasting all test x all train at once -> failed due to memory explosion

In [ ]:
test_data = test_dataset.data.view(len(test_dataset), -1).float().to(device)
train_data = train_dataset.data.view(len(train_dataset), -1).float().to(device)

# Broadcasting try
dist_broadcast_all = torch.sqrt(torch.sum((test_data.unsqueeze(1) - train_data) ** 2, dim=2))
print(dist_broadcast_all.shape)


In [10]:
import heapq

device = torch.device("cpu")

train_data_cpu = train_data.view(len(train_data), -1).to(device)
train_labels_cpu = train_labels.to(device)
test_data_cpu = test_data.view(len(test_data), -1).to(device)
test_labels_cpu = test_labels.to(device)

result_all = []

for i in range(len(test_data_cpu)):
    test_one = test_data_cpu[i].unsqueeze(0)

    if i % 500 == 0:
        print(f"Process {i}/{len(test_data_cpu)}")

    dist_broadcast = torch.sqrt(torch.sum((train_data_cpu - test_one) ** 2, dim=1))

    nearest = []
    k = 5
    for j in range(len(dist_broadcast)):
        dist = dist_broadcast[j].item()
        if len(nearest) < k:
            heapq.heappush(nearest, (-dist, train_labels_cpu[j].item()))  # Max heap
        else:
            if dist < -nearest[0][0]:  # 가장 먼 이웃보다 가까우면 교체
                heapq.heappop(nearest)
                heapq.heappush(nearest, (-dist, train_labels_cpu[j].item()))

    neighbor_labels = torch.tensor([label for _, label in nearest])
    result = torch.mode(neighbor_labels, dim=0).values.item()
    result_all.append(result)

result_all_tensor = torch.tensor(result_all)
differ = result_all_tensor - test_labels_cpu

correct_num = (differ == 0).sum().item()
accuracy = correct_num / len(test_labels_cpu) * 100

print(f"Results for all images: {result_all}")
print(f"Classification accuracy: {accuracy:.3f}%")


Process 0/10000
Process 500/10000
Process 1000/10000
Process 1500/10000
Process 2000/10000
Process 2500/10000
Process 3000/10000
Process 3500/10000
Process 4000/10000
Process 4500/10000
Process 5000/10000
Process 5500/10000
Process 6000/10000
Process 6500/10000
Process 7000/10000
Process 7500/10000
Process 8000/10000
Process 8500/10000
Process 9000/10000
Process 9500/10000
Classification accuracy: 96.880%


## **(d)** If there is any issue from (c), what is the cause of the issue? Improve the algorithm from (c) by resolving the issue you find from (c)


In [4]:
test_data = test_dataset.data.view(len(test_dataset), -1).float().to(device)
test_labels = test_dataset.targets.to(device)

# torch.cdist() → 두 텐서 간의 pairwise 거리 계산 (p=2는 Euclidean 거리)
dist_broadcast_all = torch.cdist(test_data, train_data, p=2)  # (10000, 60000)

result_all = []
k = 5

# test 샘플마다 최근접 이웃 찾기
for i in range(len(test_data)):
    dist = dist_broadcast_all[i]

    # torch.topk()로 k개의 최근접 이웃 인덱스 선택
    _, nearest_index = torch.topk(dist, k, largest=False)

    # 해당 이웃들의 라벨 추출
    neighbor_labels = train_labels[nearest_index]

    # torch.mode()로 최빈값 라벨 결정
    result = torch.mode(neighbor_labels, dim=0).values.item()
    result_all.append(result)

# 정확도 계산
result_all_tensor = torch.tensor(result_all, device=device)
differ = result_all_tensor - test_labels

correct_num = (differ == 0).sum().item()
accuracy = correct_num / len(test_labels) * 100

print(f"Results for all images: {result_all[:50]} ...")  # 앞 50개만 출력
print(f"Classification accuracy: {accuracy:.3f}%")

Results for all images: [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 0, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4] ...
Classification accuracy: 96.880%


### (f) Try at least two other options for each hyperparameter. Write down your observations.

In [5]:
test_data = test_dataset.data.view(len(test_dataset), -1).float().to(device)
test_labels = test_dataset.targets.to(device)

dist_broadcast_all = torch.cdist(test_data, train_data, p=2)  # (10000, 60000)

result_all = []
k = 2

for i in range(len(test_data)):
    dist = dist_broadcast_all[i]

    _, nearest_index = torch.topk(dist, k, largest=False)

    neighbor_labels = train_labels[nearest_index]

    result = torch.mode(neighbor_labels, dim=0).values.item()
    result_all.append(result)

result_all_tensor = torch.tensor(result_all, device=device)
differ = result_all_tensor - test_labels

correct_num = (differ == 0).sum().item()
accuracy = correct_num / len(test_labels) * 100

print(f"Results for all images: {result_all[:50]} ...")
print(f"Classification accuracy: {accuracy:.3f}%")

Results for all images: [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 0, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4] ...
Classification accuracy: 96.270%


In [6]:
test_data = test_dataset.data.view(len(test_dataset), -1).float().to(device)
test_labels = test_dataset.targets.to(device)

dist_broadcast_all = torch.cdist(test_data, train_data, p=2)  # (10000, 60000)

result_all = []
k = 232

for i in range(len(test_data)):
    dist = dist_broadcast_all[i]

    _, nearest_index = torch.topk(dist, k, largest=False)

    neighbor_labels = train_labels[nearest_index]

    result = torch.mode(neighbor_labels, dim=0).values.item()
    result_all.append(result)

result_all_tensor = torch.tensor(result_all, device=device)
differ = result_all_tensor - test_labels

correct_num = (differ == 0).sum().item()
accuracy = correct_num / len(test_labels) * 100

print(f"Results for all images: {result_all[:50]} ...")  # 앞 50개만 출력
print(f"Classification accuracy: {accuracy:.3f}%")

Results for all images: [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 6, 7, 2, 7, 1, 1, 1, 1, 7, 4, 1, 3, 5, 1, 2, 4, 4] ...
Classification accuracy: 92.490%


## **(g)** You can try more options if you want. What is the final test accuracy?

In [7]:
test_data = test_dataset.data.view(len(test_dataset), -1).float().to(device)
test_labels = test_dataset.targets.to(device)

dist_broadcast_all = torch.cdist(test_data, train_data, p=2)  # (10000, 60000)

result_all = []
k = 10

for i in range(len(test_data)):
    dist = dist_broadcast_all[i]

    _, nearest_index = torch.topk(dist, k, largest=False)

    neighbor_labels = train_labels[nearest_index]

    result = torch.mode(neighbor_labels, dim=0).values.item()
    result_all.append(result)

result_all_tensor = torch.tensor(result_all, device=device)
differ = result_all_tensor - test_labels

correct_num = (differ == 0).sum().item()
accuracy = correct_num / len(test_labels) * 100

print(f"Results for all images: {result_all[:50]} ...")
print(f"Classification accuracy: {accuracy:.3f}%")

Results for all images: [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 0, 7, 2, 7, 1, 2, 1, 1, 7, 4, 1, 3, 5, 1, 2, 4, 4] ...
Classification accuracy: 96.650%


In [8]:
test_data = test_dataset.data.view(len(test_dataset), -1).float().to(device)
test_labels = test_dataset.targets.to(device)

dist_broadcast_all = torch.cdist(test_data, train_data, p=1)

result_all = []
k = 5

for i in range(len(test_data)):
    dist = dist_broadcast_all[i]

    _, nearest_index = torch.topk(dist, k, largest=False)

    neighbor_labels = train_labels[nearest_index]

    result = torch.mode(neighbor_labels, dim=0).values.item()
    result_all.append(result)

result_all_tensor = torch.tensor(result_all, device=device)
differ = result_all_tensor - test_labels

correct_num = (differ == 0).sum().item()
accuracy = correct_num / len(test_labels) * 100

print(f"Results for all images: {result_all[:50]} ...")
print(f"Classification accuracy: {accuracy:.3f}%")

Results for all images: [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 9, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 1, 3, 5, 1, 2, 4, 4] ...
Classification accuracy: 96.180%


In [9]:
test_data = test_dataset.data.view(len(test_dataset), -1).float().to(device)
test_labels = test_dataset.targets.to(device)

dist_broadcast_all = torch.cdist(test_data, train_data, p=1)  # (10000, 60000)

result_all = []
k = 10

for i in range(len(test_data)):
    dist = dist_broadcast_all[i]

    _, nearest_index = torch.topk(dist, k, largest=False)

    neighbor_labels = train_labels[nearest_index]

    result = torch.mode(neighbor_labels, dim=0).values.item()
    result_all.append(result)

result_all_tensor = torch.tensor(result_all, device=device)
differ = result_all_tensor - test_labels

correct_num = (differ == 0).sum().item()
accuracy = correct_num / len(test_labels) * 100

print(f"Results for all images: {result_all[:50]} ...")
print(f"Classification accuracy: {accuracy:.3f}%")

Results for all images: [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 1, 1, 1, 7, 4, 1, 3, 5, 1, 2, 4, 4] ...
Classification accuracy: 95.890%


In [10]:
test_data = test_dataset.data.view(len(test_dataset), -1).float().to(device)
test_labels = test_dataset.targets.to(device)

dist_broadcast_all = torch.cdist(test_data, train_data, p=3)  # (10000, 60000)

result_all = []
k = 5

for i in range(len(test_data)):
    dist = dist_broadcast_all[i]

    _, nearest_index = torch.topk(dist, k, largest=False)

    neighbor_labels = train_labels[nearest_index]

    result = torch.mode(neighbor_labels, dim=0).values.item()
    result_all.append(result)

result_all_tensor = torch.tensor(result_all, device=device)
differ = result_all_tensor - test_labels

correct_num = (differ == 0).sum().item()
accuracy = correct_num / len(test_labels) * 100

print(f"Results for all images: {result_all[:50]} ...")
print(f"Classification accuracy: {accuracy:.3f}%")

Results for all images: [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 0, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4] ...
Classification accuracy: 97.190%


### Weighted voting

In [13]:
device = torch.device("cpu")

test_data = test_dataset.data.view(len(test_dataset), -1).float().to(device)
test_labels = test_dataset.targets.to(device)

dist_broadcast_all = torch.cdist(test_data, train_data, p=2)  # (10000, 60000)

k = 5
result_all = []

for i in range(len(test_data)):  # test_data_cpu → test_data
    dist = dist_broadcast_all[i]

    topk_dist, topk_indices = torch.topk(dist, k, largest=False)
    neighbor_labels = train_labels[topk_indices]  # train_labels_cpu → train_labels

    weights = 1 / (topk_dist + 1e-8)

    weighted_sum = {}
    for lbl, w in zip(neighbor_labels.tolist(), weights.tolist()):
        weighted_sum[lbl] = weighted_sum.get(lbl, 0) + w

    predicted_label = max(weighted_sum, key=weighted_sum.get)
    result_all.append(predicted_label)

result_all_tensor = torch.tensor(result_all)
correct_num = (result_all_tensor == test_labels).sum().item()  # test_labels_cpu → test_labels
accuracy = correct_num / len(test_labels) * 100  # test_labels_cpu → test_labels

print(f"Results for first 10 images: {result_all[:10]} ...")
print(f"Classification accuracy (Weighted Voting): {accuracy:.3f}%")

Results for first 10 images: [7, 2, 1, 0, 4, 1, 4, 9, 5, 9] ...
Classification accuracy (Weighted Voting): 96.910%


In [14]:
device = torch.device("cpu")

test_data = test_dataset.data.view(len(test_dataset), -1).float().to(device)
test_labels = test_dataset.targets.to(device)

dist_broadcast_all = torch.cdist(test_data, train_data, p=2)  # (10000, 60000)

k = 10
result_all = []

for i in range(len(test_data)):  # test_data_cpu → test_data
    dist = dist_broadcast_all[i]

    topk_dist, topk_indices = torch.topk(dist, k, largest=False)
    neighbor_labels = train_labels[topk_indices]  # train_labels_cpu → train_labels

    weights = 1 / (topk_dist + 1e-8)

    weighted_sum = {}
    for lbl, w in zip(neighbor_labels.tolist(), weights.tolist()):
        weighted_sum[lbl] = weighted_sum.get(lbl, 0) + w

    predicted_label = max(weighted_sum, key=weighted_sum.get)
    result_all.append(predicted_label)

result_all_tensor = torch.tensor(result_all)
correct_num = (result_all_tensor == test_labels).sum().item()  # test_labels_cpu → test_labels
accuracy = correct_num / len(test_labels) * 100  # test_labels_cpu → test_labels

print(f"Results for first 10 images: {result_all[:10]} ...")
print(f"Classification accuracy (Weighted Voting): {accuracy:.3f}%")

Results for first 10 images: [7, 2, 1, 0, 4, 1, 4, 9, 5, 9] ...
Classification accuracy (Weighted Voting): 96.840%


In [15]:
device = torch.device("cpu")

test_data = test_dataset.data.view(len(test_dataset), -1).float().to(device)
test_labels = test_dataset.targets.to(device)

dist_broadcast_all = torch.cdist(test_data, train_data, p=2)  # (10000, 60000)

k = 20
result_all = []

for i in range(len(test_data)):  # test_data_cpu → test_data
    dist = dist_broadcast_all[i]

    topk_dist, topk_indices = torch.topk(dist, k, largest=False)
    neighbor_labels = train_labels[topk_indices]  # train_labels_cpu → train_labels

    weights = 1 / (topk_dist + 1e-8)

    weighted_sum = {}
    for lbl, w in zip(neighbor_labels.tolist(), weights.tolist()):
        weighted_sum[lbl] = weighted_sum.get(lbl, 0) + w

    predicted_label = max(weighted_sum, key=weighted_sum.get)
    result_all.append(predicted_label)

result_all_tensor = torch.tensor(result_all)
correct_num = (result_all_tensor == test_labels).sum().item()  # test_labels_cpu → test_labels
accuracy = correct_num / len(test_labels) * 100  # test_labels_cpu → test_labels

print(f"Results for first 10 images: {result_all[:10]} ...")
print(f"Classification accuracy (Weighted Voting): {accuracy:.3f}%")

Results for first 10 images: [7, 2, 1, 0, 4, 1, 4, 9, 5, 9] ...
Classification accuracy (Weighted Voting): 96.330%
